# Fine-tune masked ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте precorrect.py

In [2]:
with open('CorrectDN.json', encoding="utf-8") as f:
    sentences = pd.read_json(f, orient='records')
sentences.tail()

,text,label
1042,"В Саратове при пожаре погиб мужчина, спасшийся...",1
1043,Лаврентьев: США приказали оппозиции в Сирии на...,1
1044,Лаврентьев рассказал о контактах спецслужб Рос...,1
1045,В Ростове-на-Дону арестовали двух украинских а...,1
1046,Посол США обвинил Орбана в пророссийской внешн...,1


In [3]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/1047 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1047 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['1'], id=None)}

Datasets:

In [4]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [5]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "cointegrated/rubert-tiny2"
# model_checkpoint = "DeepPavlov/rubert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [39]:
text = "После начала российской военной [MASK] на Украине западные страны усилили санкционное давление на Москву"

In [40]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
#print (inputs)
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
# print (mask_token_index)
mask_token_logits = token_logits[0, mask_token_index, :]

# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

word = "спецоперации"
iii = tokenizer(word, return_tensors="pt")
moscow_token = iii["input_ids"][0][1].item()
print (moscow_token)

probs = torch.nn.functional.softmax(mask_token_logits, dim=1)
print (probs[0][29571])
print (probs[0][19534])
print (probs[0][moscow_token])

top_5_prob = torch.topk(probs, 5, dim=1)
val = top_5_prob.values[0].tolist()
ind = top_5_prob.indices[0].tolist()
for i in range(len(val)):
    print (f"'>>> {tokenizer.decode([ind[i]])} ({ind[i]})  >>>>: {val[i]}")
print ("-----------------")



# for i in range(len(val_all)):
#     if ind[i] == 17627:
#         print (f"'>>> {tokenizer.decode([ind_all[i]])}  >>>>: {val_all[i]}")

#for token in top_5_tokens:
#    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}' {token}")

61407
tensor(4.6802e-07, grad_fn=<SelectBackward0>)
tensor(2.7577e-05, grad_fn=<SelectBackward0>)
tensor(7.1256e-07, grad_fn=<SelectBackward0>)
'>>> службы (10969)  >>>>: 0.10826659202575684
'>>> операции (11394)  >>>>: 0.07997777312994003
'>>> политики (17480)  >>>>: 0.07752140611410141
'>>> силы (11885)  >>>>: 0.06181306019425392
'>>> кампании (22938)  >>>>: 0.046161551028490067
-----------------


In [10]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# Use batched=True to activate fast multithreading!
tokenized_datasets = raw_ds.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/553 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 553
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 139
    })
})

In [11]:
tokenizer.model_max_length

2048

In [12]:
chunk_size = 128

In [13]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 59'
'>>> Review 1 length: 481'
'>>> Review 2 length: 158'


In [14]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 698'


In [15]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 58'


In [16]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [17]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/553 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 234
    })
})

In [18]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'##ХАЛ ). Военные добавили, что на месте происшествия были обнаружены винтовка, топор и боеприпасы. Израиль 7 октября подвергся беспрецедентной по масштабу ракетной атаке из сектора Газа в рамках объявленной военным крылом палестинского движения ХАМАС операции " Потоп Аль - Аксы ". После этого бойцы организации проникли в приграничные районы на юге Израиля, где открывали огонь как по военным, так и по гражданским, а также взяли более 200 заложников. В Израиле, по последним данным властей, погибли около 1, 2 тысячи человек, это число включает гражданских лиц, солдат, иностранных граждан и рабочих, также сообщалось, что пострадали более 5 тысяч. В ответ'

In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [20]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] [MASK]ин рассказал о росте раскрытых дел об экстремизме председатель СК [MASK] Александр Бастрыкин Ансам " Что касается экстремизма, то мы направили в суд 430 уголовных дел, что на 62 % больше, чем в прошлом году ", - сказал глава ведомства. [MASK] полный текст интервью > > > [SEP] [CLS] [MASK] ЦАХАЛ застрелили палестин [MASK] на блокпосту на Западном берегу ТЕЛЬ - [MASK]ИВ [MASK] 14 янв [MASK] [MASK] [MASK]. Израильские [MASK] в воскресенье застрелили палестинцев, которые на [MASK] прорвались [MASK] один из блокпостов на [MASK] [MASK] и во [MASK] погони стреляли по солдатам, [MASK] пресс - служба Армии обороны Израиля ( ЦА'

'>>> ##ХАЛ [MASK]. Военные добавили, что на месте [MASK] были обнаружены винтов [MASK], [MASK] и боеприпасы [MASK] Израиль 7 октября подвергся беспрецедент [MASK] по масштабу ракетной атаке из сектора Газа в рамках объявленной военным крылом палестинского движения ХАМАС операции [MASK] Потоп [MASK] - [MASK]сы [MASK]. После этого бойцы организации [MASK

In [21]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [25]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] Бастрыкин рассказал о росте раскрытых дел об экстремизме председатель СК РФ Александр Бастрыкин. [MASK] Что касается экстремизма, то [MASK] направили в [MASK] [MASK] уголовных дел, что на [MASK] % больше, [MASK] в прошлом году ", - сказал [MASK] ведомства. [MASK] полный текст интервью > > > [SEP] [CLS] Военные ЦАХАЛ застрелили палестинцев на блокпосту на Западном берегу [MASK] [MASK] [MASK] АВИВ, 14 янв - РИА Новости. [MASK] [MASK] [MASK] в [MASK] застрелили палестинцев, [MASK] [MASK] автомобиле прорвались через один из блокпостов на Западном берегу и во время погони стреляли по солдатам, [MASK] пресс - служба Армии обороны Израиля ( ЦА'

'>>> ##ХАЛ ). Военные добавили, [MASK] на месте происшествия были [MASK] винтовка [MASK] топор и [MASK] [MASK] Израиль [MASK] октября подвергся беспрецедентной по масштабу ракетной [MASK] из сектора Газа в рамках объявленной военным крылом палестинского движения ХАМАС операции " Потоп Аль [MASK] Аксы [MASK]. После этого бойцы организации п

In [31]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(lm_datasets["train"]) // batch_size

training_args = TrainingArguments(
    output_dir="test-trainer",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # push_to_hub=True,
    # fp16=True,
    logging_steps=logging_steps,
)

In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [33]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/4 [00:24<?, ?it/s]

>>> Perplexity: 30.19
